In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation, neighbors, metrics, linear_model, ensemble, grid_search, cluster
import matplotlib.pyplot as plt

df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(df.head())
#print(test.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [4]:
import statistics as s
#print(s.mean(df.Age))
#print(df.describe())
m = df.describe().transpose()['mean']
thq = df.describe().transpose()['75%']
#print(m.Age)

df.Age = df.Age.fillna(value=m.Age)
df['Child'] = np.where(df['Age'] < 18, 1, 0)

print(df.head())

mtest = test.describe().transpose()['mean']
test.Age = test.Age.fillna(value=mtest.Age)
test['Child'] = np.where(test['Age'] < 18, 1, 0)


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  Child  
0      0         A/5 21171   7.2500   NaN        S      0  
1      0          PC 17599  71.2833   C85        C      0  
2      0  STON/O2. 3101282   7.9250   NaN        S      0  
3      0            113803  53.1000  C123        S      0  
4      0            

/Users/avinashtamby/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


In [5]:
#Feature Engineering Training

import re
title = []
for i in range(len(df.Name)):
    title.append(re.split(', |\. ',df.Name[i])[1])

print(title)

df['Title'] = title
df.Title.value_counts()

for i in range(len(df.Title)):
    if df.Title[i] == 'Mme':
        df.Title[i] = 'Mlle'
    elif df.Title[i] in ('Capt','Don','Major'):
        df.Title[i] = 'Sir'
    elif df.Title[i] in ('Dona','the Countess','Jonkheer'):
        df.Title[i] = 'Lady'
df.Title.value_counts()

df['FamilySize'] = df.SibSp + df.Parch + 1
print(df.FamilySize)

surname = []
for i in range(len(df.Name)):
    surname.append(re.split(', |\. ',df.Name[i])[0])

#print(surname)
df['Surname'] = surname
df.Surname

familyid = []
for i in range(len(df)):
    familyid.append(str(df.FamilySize[i])+df.Surname[i])

df['FamilyID'] = familyid
df.FamilyID

['Mr', 'Mrs', 'Miss', 'Mrs', 'Mr', 'Mr', 'Mr', 'Master', 'Mrs', 'Mrs', 'Miss', 'Miss', 'Mr', 'Mr', 'Miss', 'Mrs', 'Master', 'Mr', 'Mrs', 'Mrs', 'Mr', 'Mr', 'Miss', 'Mr', 'Miss', 'Mrs', 'Mr', 'Mr', 'Miss', 'Mr', 'Don', 'Mrs', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Miss', 'Miss', 'Mrs', 'Mrs', 'Mr', 'Miss', 'Miss', 'Mr', 'Mr', 'Miss', 'Mr', 'Mrs', 'Master', 'Mr', 'Mrs', 'Mrs', 'Mr', 'Mr', 'Miss', 'Mr', 'Miss', 'Master', 'Mr', 'Miss', 'Mr', 'Master', 'Mr', 'Master', 'Mrs', 'Mr', 'Miss', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Master', 'Miss', 'Mr', 'Mr', 'Miss', 'Mr', 'Miss', 'Mrs', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mrs', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Miss', 'Mr', 'Miss', 'Mr', 'Miss', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Miss', 'Mr', 'Master', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Mr', 'Mr', 'Miss', 'Mr', 'Mr', 'Mr', 'Mrs', 'Miss', 'Mrs', 'Mr', 'Mr', 'Mr', 'Mr

/Users/avinashtamby/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/avinashtamby/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/avinashtamby/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0       2
1       2
2       1
3       2
4       1
5       1
6       1
7       5
8       3
9       2
10      3
11      1
12      1
13      7
14      1
15      1
16      6
17      1
18      2
19      1
20      1
21      1
22      1
23      1
24      5
25      7
26      1
27      6
28      1
29      1
       ..
861     2
862     1
863    11
864     1
865     1
866     2
867     1
868     1
869     3
870     1
871     3
872     1
873     1
874     2
875     1
876     1
877     1
878     1
879     2
880     2
881     1
882     1
883     1
884     1
885     6
886     1
887     1
888     4
889     1
890     1
Name: FamilySize, dtype: int64


0               2Braund
1              2Cumings
2            1Heikkinen
3             2Futrelle
4                1Allen
5                1Moran
6             1McCarthy
7              5Palsson
8              3Johnson
9               2Nasser
10           3Sandstrom
11             1Bonnell
12         1Saundercock
13           7Andersson
14             1Vestrom
15             1Hewlett
16                6Rice
17            1Williams
18       2Vander Planke
19          1Masselmani
20              1Fynney
21             1Beesley
22             1McGowan
23              1Sloper
24             5Palsson
25             7Asplund
26                1Emir
27             6Fortune
28             1O'Dwyer
29            1Todoroff
             ...       
861              2Giles
862              1Swift
863              11Sage
864               1Gill
865            1Bystrom
866       2Duran y More
867           1Roebling
868      1van Melkebeke
869            3Johnson
870             1Balkic
871           3B

In [6]:
#Feature Engineering Test

testtitle = []
for i in range(len(test)):
    testtitle.append(re.split(', |\. ',test.Name[i])[1])

test['Title'] = testtitle
test.Title.value_counts()

for i in range(len(test.Title)):
    if test.Title[i] == 'Mme':
        test.Title[i] = 'Mlle'
    elif test.Title[i] in ('Capt','Don','Major'):
        test.Title[i] = 'Sir'
    elif test.Title[i] in ('Dona','the Countess','Jonkheer'):
        test.Title[i] = 'Lady'

test['FamilySize'] = test.SibSp + test.Parch + 1

testsurname = []
for i in range(len(test)):
    testsurname.append(re.split(', |\. ',test.Name[i])[0])

test['Surname'] = testsurname

testfamilyid = []
for i in range(len(test)):
    testfamilyid.append(str(test.FamilySize[i])+test.Surname[i])

test['FamilyID'] = testfamilyid

/Users/avinashtamby/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
for i in range(len(df)):
    if pd.isnull(df.Embarked[i]):
        df.Embarked[i] = 'U'

df.Sex = preprocessing.LabelEncoder().fit_transform(df.Sex) #Encode training data
df.Title = preprocessing.LabelEncoder().fit_transform(df.Title)
df.Embarked = preprocessing.LabelEncoder().fit_transform(df.Embarked)

test.Sex = preprocessing.LabelEncoder().fit_transform(test.Sex)
test.Title = preprocessing.LabelEncoder().fit_transform(test.Title)
test.Embarked = preprocessing.LabelEncoder().fit_transform(test.Embarked)

X = np.array(df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title','FamilySize']])
print(X[0:5])
print(X.shape)

y = np.array(df['Survived'])

print(y[0:5])
print(y.shape)

[[  3.       1.      22.       1.       0.       7.25     2.       6.       2.    ]
 [  1.       0.      38.       1.       0.      71.2833   0.       7.       2.    ]
 [  3.       0.      26.       0.       0.       7.925    2.       4.       1.    ]
 [  1.       0.      35.       1.       0.      53.1      2.       7.       2.    ]
 [  3.       1.      35.       0.       0.       8.05     2.       6.       1.    ]]
(891, 9)
[0 1 1 1 0]
(891,)


/Users/avinashtamby/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
kmn = cluster.KMeans(n_clusters=2)
kmn.fit(X)

TP = 0
for i in range(len(y)):
    if kmn.predict(X)[i] == y[i]:
        TP+=1
accuracy = TP/len(y)
print(accuracy)

0.6442199775533108


In [ ]:
#knn = neighbors.KNeighborsClassifier(n_neighbors=7,n_jobs=-1)
#svm = svm.SVC()
#logit = linear_model.LogisticRegression(n_jobs=-1)
clf = ensemble.RandomForestClassifier(n_jobs=-1)

k_range = list(range(1,50))
param_dist = dict(n_estimators=k_range)
rand = grid_search.RandomizedSearchCV(clf, param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5, n_jobs=-1)
rand.fit(X, y)
rand.grid_scores_
print(rand.best_score_, rand.best_params_)

In [ ]:
rf = ensemble.RandomForestClassifier(n_jobs=-1)

def classifier(alg, features):
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, y, test_size=0.1)
    alg.fit(X_train, y_train)
    alg_train_pp = alg.predict_proba(X_test)
    print(alg.score(X_test, y_test))
    
    #ROC Curve
    fpr, tpr, thresholds = metrics.roc_curve(y_test, alg_train_pp[:,1])
    roc_auc = metrics.auc(fpr, tpr)
    %matplotlib inline
    fig = plt.plot(fpr, tpr, label='AUC = %0.3f' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate or (1 - Specificity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
    return fig

In [ ]:
classifier(rf,X)